In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import math
import requests
import lxml
import re
import time
from fake_useragent import UserAgent

<IPython.core.display.Javascript object>

In [8]:
area_dic = {
    "工业园区": "gongyeyuan",
    "吴中区": "wuzhong",
    "姑苏区": "gusu",
    # "高新区": "gaoxin1",
    "相城区": "xiangcheng",
    "吴江区": "wujiang",
    "昆山市": "kunshan",
}

<IPython.core.display.Javascript object>

In [10]:
ua = UserAgent()

<IPython.core.display.Javascript object>

In [11]:
headers = {
    "User-Agent": ua.random,
    "Referer": "https://su.lianjia.com/ershoufang/",
}

<IPython.core.display.Javascript object>

In [12]:
sess = requests.session()
sess.get("https://su.lianjia.com/ershoufang/", headers=headers)

url = "https://su.lianjia.com/ershoufang/{}/pg{}/"

<IPython.core.display.Javascript object>

In [13]:
def re_match(re_pattern, string, errif=None):
    try:
        return re.findall(re_pattern, string)[0].strip()
    except IndexError:
        return errif

<IPython.core.display.Javascript object>

In [14]:
# 新建一个DataFrame存储信息
data = pd.DataFrame()

for key_, value_ in area_dic.items():
    # 获取该行政区下房源记录数
    start_url = "https://su.lianjia.com/ershoufang/{}/".format(value_)
    html = sess.get(start_url).text
    house_num = re.findall("共找到<span> (.*?) </span>套.*二手房", html)[0].strip()
    print("💚{}: 二手房源共计「{}」套".format(key_, house_num))
    time.sleep(1)
    # 页面限制🚫 每个行政区只能获取最多100页共计3000条房源信息
    total_page = int(math.ceil(min(3000, int(house_num)) / 30.0))
    for i in tqdm(range(total_page), desc=key_):
        html = sess.get(url.format(value_, i + 1)).text
        soup = BeautifulSoup(html, "lxml")
        info_collect = soup.find_all(class_="info clear")

        for info in info_collect:
            info_dic = {}
            # 行政区
            info_dic["area"] = key_
            # 房源的标题
            info_dic["title"] = re_match('target="_blank">(.*?)</a><!--', str(info))
            # 小区名
            info_dic["community"] = re_match(
                'xiaoqu.*?target="_blank">(.*?)</a>', str(info)
            )
            # 位置
            info_dic["position"] = re_match(
                '<a href.*?target="_blank">(.*?)</a>.*?class="address">', str(info)
            )
            # 税相关，如房本满5年
            info_dic["tax"] = re_match('class="taxfree">(.*?)</span>', str(info))
            # 总价
            info_dic["total_price"] = float(
                re_match('class="totalPrice"><span>(.*?)</span>万', str(info))
            )
            # 单价
            info_dic["unit_price"] = float(re_match('data-price="(.*?)"', str(info)))

            # 匹配房源标签信息，通过|切割
            # 包括面积，朝向，装修等信息
            icons = (
                re.findall('class="houseIcon"></span>(.*?)</div>', str(info))[0]
                .strip()
                .split("|")
            )
            info_dic["hourseType"] = icons[0].strip()
            info_dic["hourseSize"] = float(icons[1].replace("平米", ""))
            info_dic["direction"] = icons[2].strip()
            info_dic["fitment"] = icons[3].strip()

            # 存入DataFrame
            if data.empty:
                data = pd.DataFrame(info_dic, index=[0])
            else:
                data = data.append(info_dic, ignore_index=True)

💚工业园区: 二手房源共计「15235」套


工业园区: 100%|██████████████████████████████████████████████████████████████████████| 100/100 [01:44<00:00,  1.05s/it]


💚吴中区: 二手房源共计「15871」套


吴中区: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [01:49<00:00,  1.10s/it]


💚姑苏区: 二手房源共计「11971」套


...

💚吴江区: 二手房源共计「9231」套


吴江区: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [02:08<00:00,  1.28s/it]


💚昆山市: 二手房源共计「26843」套


昆山市: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [02:25<00:00,  1.46s/it]


<IPython.core.display.Javascript object>

In [15]:
data

,area,title,community,position,tax,total_price,unit_price,hourseType,hourseSize,direction,fitment
0,工业园区,金湖湾正四房 四开间朝南 开发商统一装修,金湖湾花园,湖东CBD,房本满五年,825.0,49771.0,4室2厅,165.76,南,精装
1,工业园区,玲珑湾复式四房 利用高 双开门 诚售 看房有钥匙,玲珑湾花园二区,玲珑,房本满五年,788.0,46255.0,4室2厅,170.36,南 北,毛坯
2,工业园区,户型方正，楼层采光好，房东诚意出售，看房方便,东湖春之韵,湖东CBD,房本满五年,519.0,43894.0,3室2厅,118.24,南,精装
3,工业园区,尚玲珑湾大平层四房 四开间朝南 小区中间位置安静,玲珑湾花园七八区,玲珑,None,1280.0,67178.0,4室2厅,190.54,东,精装
4,工业园区,奥体板块品质两房，户型正气，南北通风，位置采光好,中海国际社区（四区）,湖东,房本满五年,418.0,43093.0,2室2厅,97.00,南,精装
...,...,...,...,...,...,...,...,...,...,...,...
17994,昆山市,雅居乐二期前排，烫金楼层，4房，新城公园商圈物业好,昆山雅居乐花园,千灯,None,160.0,13331.0,4室2厅,120.03,南 北,精装
17995,昆山市,地铁口满五唯一 南北户型 小区中间景观楼层,东方华庭,玉山老城区,房本满五年,298.0,21077.0,3室2厅,141.39,南 北,简装
17996,昆山市,万科经典两房 小房间改造好 楼层位置无遮挡 满二,万科魅力花园,花桥,None,125.0,18903.0,2室2厅,66.13,南,精装
17997,昆山市,花苑简装小两房，南北通透，中间楼层，诚心买价可谈,花苑新村（花桥）,花桥,房本满五年,126.0,15756.0,2室2厅,79.97,南 北,简装


<IPython.core.display.Javascript object>

In [17]:
data.to_excel("suzhou.xlsx", index=False)

<IPython.core.display.Javascript object>

In [18]:
ad = {"高新区": "gaoxin1"}

<IPython.core.display.Javascript object>

In [19]:
# 新建一个DataFrame存储信息
data = pd.DataFrame()

for key_, value_ in ad.items():
    # 获取该行政区下房源记录数
    start_url = "https://su.lianjia.com/ershoufang/{}/".format(value_)
    html = sess.get(start_url).text
    house_num = re.findall("共找到<span> (.*?) </span>套.*二手房", html)[0].strip()
    print("💚{}: 二手房源共计「{}」套".format(key_, house_num))
    time.sleep(1)
    # 页面限制🚫 每个行政区只能获取最多100页共计3000条房源信息
    total_page = int(math.ceil(min(3000, int(house_num)) / 30.0))
    for i in tqdm(range(total_page), desc=key_):
        html = sess.get(url.format(value_, i + 1)).text
        soup = BeautifulSoup(html, "lxml")
        info_collect = soup.find_all(class_="info clear")

        for info in info_collect:
            info_dic = {}
            # 行政区
            info_dic["area"] = key_
            # 房源的标题
            info_dic["title"] = re_match('target="_blank">(.*?)</a><!--', str(info))
            # 小区名
            info_dic["community"] = re_match(
                'xiaoqu.*?target="_blank">(.*?)</a>', str(info)
            )
            # 位置
            info_dic["position"] = re_match(
                '<a href.*?target="_blank">(.*?)</a>.*?class="address">', str(info)
            )
            # 税相关，如房本满5年
            info_dic["tax"] = re_match('class="taxfree">(.*?)</span>', str(info))
            # 总价
            info_dic["total_price"] = float(
                re_match('class="totalPrice"><span>(.*?)</span>万', str(info))
            )
            # 单价
            info_dic["unit_price"] = float(re_match('data-price="(.*?)"', str(info)))

            # 匹配房源标签信息，通过|切割
            # 包括面积，朝向，装修等信息
            icons = (
                re.findall('class="houseIcon"></span>(.*?)</div>', str(info))[0]
                .strip()
                .split("|")
            )
            info_dic["hourseType"] = icons[0].strip()
            info_dic["hourseSize"] = float(icons[1].replace("平米", ""))
            info_dic["direction"] = icons[2].strip()
            info_dic["fitment"] = icons[3].strip()

            # 存入DataFrame
            if data.empty:
                data = pd.DataFrame(info_dic, index=[0])
            else:
                data = data.append(info_dic, ignore_index=True)

💚高新区: 二手房源共计「9707」套


高新区: 100%|████████████████████████████████████████████████████████████████████████| 100/100 [01:24<00:00,  1.19it/s]


<IPython.core.display.Javascript object>

In [20]:
data.to_excel("suzhou2.xlsx", index=False) 

<IPython.core.display.Javascript object>